In [1]:
api_key = ""
channel_id = ""

In [2]:
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=api_key)

In [3]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                          part="snippet",
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        videos += res["items"]
        next_page_token = res.get("nextPageToken")
        
        if next_page_token is None:
            break
    
    return videos

In [4]:
videos = get_channel_videos(channel_id)

In [5]:
res = youtube.videos().list(id=videos[0]["snippet"]["resourceId"]["videoId"],
                           part="statistics").execute()

In [6]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

In [7]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

In [8]:
stats = get_videos_stats(video_ids)

In [9]:
publishedAt = []
title = []
viewCount = []
for i in range(len(videos)):
    publishedAt.append(videos[i]['snippet']['publishedAt'])
    title.append(videos[i]['snippet']['title'])
    viewCount.append(stats[i]['statistics']['viewCount'])

In [10]:
dict = {}
dict["publishedAt"] = publishedAt
dict["title"] = title
dict["viewCount"] = viewCount

In [11]:
import pandas as pd
df = pd.DataFrame.from_dict(dict)

In [12]:
df['viewCount'] = df['viewCount'].astype(int)

In [13]:
df.head()

,publishedAt,title,viewCount
0,2021-04-08T08:00:09Z,에이치엘비 CAR-T 상업화 최소 3년이상 앞당길 수 있다!! CAR-T세포가 활성...,4779
1,2021-04-08T01:34:11Z,에이치엘비 주주 오세훈 서울시장 당선!!,4733
2,2021-04-07T21:00:15Z,에이치엘비 진양곤 회장은 코르키로부터 확보했던 넥스트사이언스의 메자닌에 대한 콜옵션...,7035
3,2021-04-07T08:15:01Z,에이치엘비 증선위 뉴스기사가 없는 것으로 보아 열리지 않고 무기한 연기 가능성!!,7521
4,2021-04-07T03:00:11Z,에이치엘비가 뛰어든 CAR-T치료제의 부작용은 노바티스와 길리어드사이언스에서도 보이...,3165


In [14]:
len(df)

2591

# 제목에 '에이치엘비'가 없다면 Drop하기

In [16]:
df['에이치엘비'] = df['title'].str.contains('에이치엘비')

In [18]:
df = df.loc[(df['에이치엘비'] == True)]

In [19]:
len(df)

1445

# 제목에 '에이치엘비생명과학'이 있다면 Drop하기

In [21]:
df['에이치엘비생명과학'] = df['title'].str.contains('에이치엘비생명과학')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df = df.loc[df['에이치엘비생명과학'] == False]

In [24]:
len(df)

1408

# 제목에 '에이치엘비제약'이 있다면 Drop하기

In [25]:
df['에이치엘비제약'] = df['title'].str.contains('에이치엘비제약')

In [26]:
df = df.loc[(df['에이치엘비제약'] == False)]

In [27]:
len(df)

1380

# 과거 절반 데이터 Drop하기

In [28]:
df_half = df.copy()

In [29]:
df_half = df_half[:int(len(df_half)/2)]

In [30]:
len(df_half)

690

# 느낌표 있는/없는 제목의 평균 각각 구하기

In [36]:
df_half['surprise'] = df_half['title'].str.contains('!')

In [38]:
df_half.head()

,publishedAt,title,viewCount,에이치엘비,에이치엘비생명과학,에이치엘비제약,surprise
0,2021-04-08T08:00:09Z,에이치엘비 CAR-T 상업화 최소 3년이상 앞당길 수 있다!! CAR-T세포가 활성...,4779,True,False,False,True
1,2021-04-08T01:34:11Z,에이치엘비 주주 오세훈 서울시장 당선!!,4733,True,False,False,True
2,2021-04-07T21:00:15Z,에이치엘비 진양곤 회장은 코르키로부터 확보했던 넥스트사이언스의 메자닌에 대한 콜옵션...,7035,True,False,False,True
3,2021-04-07T08:15:01Z,에이치엘비 증선위 뉴스기사가 없는 것으로 보아 열리지 않고 무기한 연기 가능성!!,7521,True,False,False,True
4,2021-04-07T03:00:11Z,에이치엘비가 뛰어든 CAR-T치료제의 부작용은 노바티스와 길리어드사이언스에서도 보이...,3165,True,False,False,True


In [40]:
df_half.groupby('surprise').viewCount.mean()

surprise
False    7119.263052
True     9155.567708
Name: viewCount, dtype: float64

물음표가 있는 영상 평균 조회수는 9155회  
믈음표가 없는 영상 평균 조회수는 7119회

# t-test 하기

In [63]:
from scipy.stats import ttest_ind

느낌표와 아닌 제목 각각 분류하기

In [64]:
surprise = df_half.loc[df_half['surprise'] == True].viewCount
no_surprise = df_half.loc[df_half['surprise'] == False].viewCount

In [65]:
len(surprise)

192

In [66]:
len(no_surprise)

498

t-test : array로 t-test한다.

In [67]:
ttest_ind(surprise, no_surprise)

Ttest_indResult(statistic=4.780182923894167, pvalue=2.142745702313225e-06)

array를 list로 변환하기

In [68]:
surprise = list(surprise)
no_surprise = list(no_surprise)

t-test : 혹시나 list로 한 번 더 t-test한다. 같은 결과다.

In [69]:
ttest_ind(surprise, no_surprise)

Ttest_indResult(statistic=4.780182923894167, pvalue=2.142745702313225e-06)

# Results
물음표가 있는 영상 평균 조회수는 9155회  
믈음표가 없는 영상 평균 조회수는 7119회  
(p=2.142)  
물음표가 있는 영상 조회수가 더 높으나 통계적 유의미성을 달성하지 못했다.

# Conclusion
물음표를 사용한다. 왜냐하면 밑져야 본전이기 때문이다. 잘 되면 조회수가 더 높을 것이다. 그리고 안 돼도 조회수는 비슷한 수준일 것이다. 그러므로 통계적 유의미성은 없으나 평균 조회수가 높은 물음표를 제목에 사용한다.